# Парсинг.
Возьмем интересующий сайт, на котором можно пособирать какие-то данные (и при этом API не предоставляется).
Идеальный датасет должен иметь текстовое описание некоторого объекта и некоторую целевую переменную, соответствующую этому объекту. Например:
Сайт новостей: текстовое описание - сама новость, целевая переменная - количество просмотров новости (можно поделить на число дней с момента даты публикации, чтобы получить “среднее число просмотров в день”).
Сайт с товарами/книгами/фильмами: текстовое описание товара/книги/фильма + средний рейтинг в качестве целевой переменной.
Блоги - тексты заметок + число просмотров.
И любые другие ваши идеи, которые подходят под такой формат.
Напишем свой парсер, который будет бегать по страничкам и автоматически что-то собирать.
Не забываем, что парсинг - это ответственное мероприятие, поэтому не бомбардируйте несчастные сайты слишком частыми запросами (можно ограничить число запросов в секунду при помощи time.sleep(0.3), вставленного в теле цикла)


# 1. Ознакомление

## 1.1. Что за парсинг?

## 1.2 Сейчас познакомимся с парой определений:

* **Парсер** — это скрипт, который грабит информацию с сайта
* **Краулер** — это часть парсера, которая бродит по ссылкам
* **Краулинг** — это переход по страницам и ссылкам
* **Скрапинг** — это сбор данных со страниц
* **Парсинг** — это сразу и краулинг и скрапинг!



## 1.3. Наш первый запрос

Доступ к веб-станицам позволяет получать модуль `requests`. Подгрузим его. За компанию подгрузим ещё парочку дельных пакетов.

In [82]:
# Импортировать модуль warnings для работы с предупреждениями
import warnings
# Отфильтровать все предупреждения и не выводить их на экран
warnings.filterwarnings("ignore")


In [83]:
# Импортировать библиотеку requests для отправки запросов к веб-серверам
import requests
# Импортировать библиотеку numpy для работы с матрицами, векторами и линейной алгеброй
import numpy as np
# Импортировать библиотеку pandas для работы с табличными данными
import pandas as pd
# Импортировать библиотеку time для работы с временем и задержками
import time

Для наших благородных исследовательских целей нужно собрать данные по каждому фильму с соответствующей ему страницы. Но для начала нужно получить адреса этих страниц. Поэтому открываем основную страницу со всеми фильмами. Выглядит она следующим образом:



Со страницы всех фильмов https://www.kinopoisk.ru/lists/movies/?b=films мы  будем тащить ссылки на каждый из перечисленных фильмов. Сохраним в переменную `page_link` адрес основной страницы и откроем её при помощи библиотеки `requests`.

In [84]:
# Присвоить переменной page_link URL-адрес первой страницы сайта 
page_link = 'https://metarankings.ru/best-movies-2023/'


Не будем заходить на сервер, на питоне и используем библиотеку requests.
Это может вызвать у него некоторые подозрения относительно наших благих намерений и он решит нас безжалостно отвергнуть.
Будем работать с  [`fake-useragent`](https://pypi.python.org/pypi/fake-useragent). При вызове метода из различных кусочков будет генерироваться рандомное сочетание операционной системы, спецификаций и версии браузера, которые можно передавать в запрос:

In [85]:
# !pip install fake_useragent

In [86]:
# подгрузим один из методов этой библиотеки
from fake_useragent import UserAgent

In [87]:
UserAgent().chrome

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'

In [88]:
# Отправить GET-запрос к веб-странице, используя поддельный user-agent для браузера Chrome
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

Замечательно, наша небольшая маскировка сработала и обманутый сервер покорно выдал благословенный 200 ответ — соединение установлено и данные получены, всё чудесно! Посмотрим, что же все-таки мы получили.

In [89]:
# Получить тип объекта response с помощью функции type()
type(response)

requests.models.Response

In [90]:
# Получить содержимое ответа в виде байтов с помощью атрибута response.content
html = response.content

In [91]:
html[:1000]

b'<!DOCTYPE html>\n<html lang="ru-RU">\n<head>    \n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n<link rel="manifest" href="/manifest.json">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<title>\xd0\x9b\xd1\x83\xd1\x87\xd1\x88\xd0\xb8\xd0\xb5 \xd1\x84\xd0\xb8\xd0\xbb\xd1\x8c\xd0\xbc\xd1\x8b 2023 \xd0\xb3\xd0\xbe\xd0\xb4\xd0\xb0 - \xd0\xa1\xd0\xb0\xd0\xbc\xd1\x8b\xd0\xb5 \xd0\xb8\xd0\xbd\xd1\x82\xd0\xb5\xd1\x80\xd0\xb5\xd1\x81\xd0\xbd\xd1\x8b\xd0\xb5 \xd0\xb8 \xd0\xbf\xd0\xbe\xd0\xbf\xd1\x83\xd0\xbb\xd1\x8f\xd1\x80\xd0\xbd\xd1\x8b\xd0\xb5 \xd1\x84\xd0\xb8\xd0\xbb\xd1\x8c\xd0\xbc\xd1\x8b \xd0\xb2\xd1\x81\xd0\xb5\xd1\x85 \xd0\xb6\xd0\xb0\xd0\xbd\xd1\x80\xd0\xbe\xd0\xb2 \xd0\xb2 2023 \xe2\x80\x94 \xd1\x80\xd0\xb5\xd0\xb9\xd1\x82\xd0\xb8\xd0\xbd\xd0\xb3 \xd0\xb8 \xd0\xbe\xd1\x86\xd0\xb5\xd0\xbd\xd0\xba\xd0\xb8</title>\n<link rel="icon" href="/favicon.png" type="image/png" />\n<link rel=\'

In [92]:
html[-1000:]

b'height=\'31\'>")\n//--></script><!--/LiveInternet-->\n</div>\n</div>\n</div>\n<div class="wbody" onclick="show(\'none\')"></div>\n<div class="window">\n</div>\n<div class="popup_block">\n<form action="https://metarankings.ru/login.php?redirect_to=https%3A%2F%2Fmetarankings.ru%2Fbest-movies-2023%2F" id="loginForm" method="post">\n<div class="field">\n<label>\xd0\x98\xd0\xbc\xd1\x8f \xd0\xbf\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xb7\xd0\xbe\xd0\xb2\xd0\xb0\xd1\x82\xd0\xb5\xd0\xbb\xd1\x8f</label>\n<div class="input">\n<input type="text" name="log" value=""  id="login" /></div>\n</div>\n<div class="field">\n<a rel="nofollow" href="https://metarankings.ru/login.php?action=lostpassword" id="forgot">\xd0\x97\xd0\xb0\xd0\xb1\xd1\x8b\xd0\xbb\xd0\xb8?</a>\n<label>\xd0\x9f\xd0\xb0\xd1\x80\xd0\xbe\xd0\xbb\xd1\x8c</label>\n<div class="input">\n<input type="password" name="pwd" value="" id="pass" /></div>\n</div>\n<div class="submit">\n<button name="submit" type="submit">\xd0\x92\xd0\xbe\xd0\xb9\xd1\x82\xd0

In [93]:
len(html)

109203

In [94]:
type(html)

bytes

Выглядит неудобоваримо, как насчет сварить из этого дела что-то покрасивее? Например, красивый суп.

## 1.4. Красивый суп

Красивый суп — это совершенно волшебная библиотека, которая из сырого и необработанного HTML кода страницы выдаст вам структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием `BeautifulSoup` — скорее всего, не то, что нам нужно.
С необработанным XML кодом страницы пакет также работает (XML — это исковерканый и превращённый в диалект, с помощью своих команд, HTML). Для того, чтобы пакет корректно работал с XML разметкой, придётся в довесок ко всему нашему арсеналу установить пакет `xml`.

In [95]:
# pip install beautifulsoup4


In [96]:
from bs4 import BeautifulSoup

Передадим функции `BeautifulSoup` текст веб-страницы, которую мы недавно получили.

In [97]:
soup = BeautifulSoup(html, 'html.parser') # В опции также можно указать lxml,
                                         # если предварительно установить одноименный пакет

МЫ используем функцию BeautifulSoup() из пакета bs4, чтобы создать объект soup, который представляет собой древовидную структуру HTML-документа, содержащегося в переменной html1. Также передаем в эту функцию параметр html.parser, который указывает, какой парсер использовать для анализа HTML-документа. Парсер - это программа, которая разбивает HTML-документ на составные части и строит из них дерево.

 можно использовать другой парсер, например lxml, для анализа HTML-документа. lxml - это библиотека, которая предоставляет быстрые и гибкие парсеры для XML и HTML. Для использования lxml в качестве парсера для Beautiful Soup 4, вам нужно сначала установить его с помощью команды pip:
 pip install lxml
Затем мы можем передать параметр lxml в функцию BeautifulSoup() вместо html.parser:
 soup = BeautifulSoup(html, 'lxml')
Это изменит то, как Beautiful Soup 4 строит дерево из HTML-документа, и может повлиять на результаты поиска и фильтрации элементов. В некоторых случаях, lxml может быть быстрее и точнее, чем html.parser.

In [98]:
type(soup)

bs4.BeautifulSoup

In [99]:
print(soup.prettify()[:2000])

<!DOCTYPE html>
<html lang="ru-RU">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="/manifest.json" rel="manifest"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   Лучшие фильмы 2023 года - Самые интересные и популярные фильмы всех жанров в 2023 — рейтинг и оценки
  </title>
  <link href="/favicon.png" rel="icon" type="image/png">
   <link href="https://metarankings.ru/wp-content/themes/metarankings/style.css" id="metarankings-style-css" media="all" rel="stylesheet" type="text/css"/>
   <script src="https://metarankings.ru/wp-includes/js/jquery/jquery.js" type="text/javascript">
   </script>
   <script src="https://metarankings.ru/wp-includes/js/jquery/jquery-migrate.min.js" type="text/javascript">
   </script>
   <meta content="Здесь собраны лучшие фильмы 2023 года выпуска — Самые интересные и популярные, рейтинг которых составлялся по данным оцен

Получим что-то вот такое:
    
```
<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHkUNWUU=","queueTime":0,"applicationTime":24,"agent":""}</script>
<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(2),u=e(3),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=
```

Стало намного лучше, не правда ли? Что же лежит в переменной `soup`? Невнимательный пользователь, скорее всего, скажет,что ничего вообще не изменилось. Тем не менее, это не так. Теперь мы можем свободно бродить по HTML-дереву страницы, искать детей, родителей и вытаскивать их!

Например, можно бродить по вершинам, указывая путь из тегов.

In [100]:
soup.html.head.title

<title>Лучшие фильмы 2023 года - Самые интересные и популярные фильмы всех жанров в 2023 — рейтинг и оценки</title>

In [101]:
type(soup.html.head.title)

bs4.element.Tag

Можно вытащить из того места, куда мы забрели, текст с помощью метода `text`.

In [102]:
soup.html.head.title.text

'Лучшие фильмы 2023 года - Самые интересные и популярные фильмы всех жанров в 2023 — рейтинг и оценки'

In [103]:
type(soup.html.head.title.text)

str

Зная адрес элемента, мы сразу можем найти его. Например, можно сделать это по классу. Следующая команда должна найти элемент, который лежит внутри тега `` и имеет класс ``.

Просматриваем нашу страницу по коду
<div class="post clear">
<a href="https://metarankings.ru/film-solnce-moyo-2023/" title="«Солнце моё»" class="thumb"><img class="post-image" src="https://metarankings.ru/images/uploads/2023/03/solnce-moyo-cover-art-100x140.jpg" alt="Солнце моё"></a>
<div class="best-prod">
<div class="counter">1</div>
</div>
<a class="name" href="https://metarankings.ru/film-solnce-moyo-2023/">Солнце моё</a>
<div class="post-meta"><p>Aftersun</p></div>
<div class="post-meta"><p>16 марта 2023 года</p></div>
<div class="post-meta"><p>Шарлотта Уэллс</p></div>
<div class="post-content">Фильм «Солнце моё» рассказывает о Софи, которая вспоминает свой отдых на море, где она была со своим отцом 20 лет назад, когда ей было 11 лет. Это была история ее взросления, с трудными отношениями с разведенным отцом, первым бунтом, первой...</div>
<div class="trailer" data-video="https://metarankings.ru/film-solnce-moyo-2023/#content-trailer"></div>
<div class="ratings-graph">
<div class="graph-critic">
<div class="graph">
<div class="small-score mark-9">8.7</div> 
Рейтинг критиков
<div class="graph-body"><div style="width:87%" class="rating-graph mark-9"></div></div>
</div>
</div>
<div class="graph-critic">
<div class="graph">
<div class="small-score mark-7">7.3</div>
Зрители 
<div class="graph-body">
<div class="rating-graph mark-7" style="width:73%"></div>
</div>
</div>
</div>
</div>
</div>

In [104]:
# Найти первый тег <a> с атрибутом class="" на HTML-странице
# И присвоить его переменной obj
obj = soup.find('a', attrs = {'class':'thumb'})
# Вывести на экран объект obj
obj

<a class="thumb" href="https://metarankings.ru/film-solnce-moyo-2023/" title="«Солнце моё»"><img alt="Солнце моё" class="post-image" src="https://metarankings.ru/images/uploads/2023/03/solnce-moyo-cover-art-100x140.jpg"/></a>

In [105]:
obj.get("class")

['thumb']

In [106]:
# Найти первый тег <> с атрибутом class="" на HTML-странице
# Использовать лямбда-функцию в качестве параметра для метода soup.find()
# Лямбда-функция проверяет, что имя тега равно '' и значение атрибута class равно ['']
obj = soup.find(lambda tag: tag.name == 'a' and tag.get('class') == ['thumb'])
obj

<a class="thumb" href="https://metarankings.ru/film-solnce-moyo-2023/" title="«Солнце моё»"><img alt="Солнце моё" class="post-image" src="https://metarankings.ru/images/uploads/2023/03/solnce-moyo-cover-art-100x140.jpg"/></a>

In [107]:
obj.get("class")

['thumb']

In [108]:
obj

<a class="thumb" href="https://metarankings.ru/film-solnce-moyo-2023/" title="«Солнце моё»"><img alt="Солнце моё" class="post-image" src="https://metarankings.ru/images/uploads/2023/03/solnce-moyo-cover-art-100x140.jpg"/></a>

In [109]:
# Проверка на вхождение
if hasattr(obj, '__dict__'):
   try:
       href = vars(obj)['href']
   except KeyError:
       href = None
else:
   href = None
obj

<a class="thumb" href="https://metarankings.ru/film-solnce-moyo-2023/" title="«Солнце моё»"><img alt="Солнце моё" class="post-image" src="https://metarankings.ru/images/uploads/2023/03/solnce-moyo-cover-art-100x140.jpg"/></a>

In [110]:
obj.attrs['href']

'https://metarankings.ru/film-solnce-moyo-2023/'

Обратите внимание, что после всех этих безумных преобразований у данных поменялся тип. Теперь они `str`. Это означет, что с ними можно работать как с текстом и пускать в ход для отсеивания лишней информации регулярные выражения.

In [111]:
print("Тип данных до вытаскивания ссылки:", type(obj))
print("Тип данных после вытаскивания ссылки:", type(obj.attrs['href']))

Тип данных до вытаскивания ссылки: <class 'bs4.element.Tag'>
Тип данных после вытаскивания ссылки: <class 'str'>


Если несколько элементов на странице обладают указанным адресом, то метод `find` вернёт только самый первый.  Чтобы найти все элементы с таким адресом, нужно использовать метод `findAll`, и на выход будет выдан список. Таким образом, мы можем получить одним поиском сразу все объекты, содержащие ссылки на страницы с мемами.

In [112]:
# Найти первый тег <> с атрибутом class="" на HTML-странице
# Использовать лямбда-функцию в качестве параметра для метода soup.find()
# Лямбда-функция проверяет, что имя тега равно '' и значение атрибута class равно ['']
obj = soup.find(lambda tag: tag.name == 'a' and tag.get('class') == ['thumb'])
obj

<a class="thumb" href="https://metarankings.ru/film-solnce-moyo-2023/" title="«Солнце моё»"><img alt="Солнце моё" class="post-image" src="https://metarankings.ru/images/uploads/2023/03/solnce-moyo-cover-art-100x140.jpg"/></a>

#### Скачать можно и итерациями как пример кода
#Создать пустой список для хранения ссылок на фильмы
film_links = []
#Для каждого номера страницы от 1 до 4
for page_number in range(1, 5):
  #Собрать ссылки на фильмы с текущей страницы
  film_links += get_page_links(page_number)
#Вывести длину списка film_links
print(len(film_links))



#### Целый код
#Импортировать модуль time для использования функции слип
import time

#Для каждого номера страницы от 1 до 4
for page_number in range(1, 5):
  #Составить ссылку на страницу поиска
  page_link = f'https://metarankings.ru/best-movies-2023/page/{page_number}'
  #Попытаться отправить GET-запрос к веб-странице
  try:
    response = requests.get(page_link)
    # Получить HTML-код страницы
    html = response.text
    # Создать объект BeautifulSoup для анализа HTML-кода
    soup = BeautifulSoup(html, 'html.parser')
    # Найти все ссылки на фильмы с атрибутом class="thumb" и добавить их в список film_links
    film_links += soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['thumb'])
  #Если возникнет исключение, например, ошибка соединения или неверный URL
  except Exception as e:
    # Вывести сообщение об ошибке и продолжить цикл
    print(f'Error while parsing page {page_number}: {e}')
    continue
  #Добавить задержку в 0.1 секунды между запросами к сайту
  time.sleep(0.1)


##### Скачиваем все ссылки

In [113]:
film_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['thumb'])
film_links[:3]

[<a class="thumb" href="https://metarankings.ru/film-solnce-moyo-2023/" title="«Солнце моё»"><img alt="Солнце моё" class="post-image" src="https://metarankings.ru/images/uploads/2023/03/solnce-moyo-cover-art-100x140.jpg"/></a>,
 <a class="thumb" href="https://metarankings.ru/bednye-neschastnye-2023/" title="Фильм «Бедные-несчастные»"><img alt="Бедные-несчастные" class="post-image" src="https://metarankings.ru/images/uploads/2023/09/bednye-neschastnye-cover-art-100x140.jpg"/></a>,
 <a class="thumb" href="https://metarankings.ru/dzhon-uik-4/" title="Фильм «Джон Уик 4»"><img alt="Джон Уик 4" class="post-image" src="https://metarankings.ru/images/uploads/2023/03/dzhon-uik-4-boxart-cover-100x140.jpg"/></a>]

In [114]:
# Ни чё себе их 50
len(film_links)

50

Осталось очистить полученный список от мусора:

In [115]:
film_links = [link.attrs['href'] for link in film_links]

In [116]:
film_links [:10]

['https://metarankings.ru/film-solnce-moyo-2023/',
 'https://metarankings.ru/bednye-neschastnye-2023/',
 'https://metarankings.ru/dzhon-uik-4/',
 'https://metarankings.ru/film-strazhi-galaktiki-chast-3-2023/',
 'https://metarankings.ru/vonka-2023/',
 'https://metarankings.ru/film-posle-yanga-2023/',
 'https://metarankings.ru/ostavlennye-2023/',
 'https://metarankings.ru/film-po-pravilam-i-bez-2023/',
 'https://metarankings.ru/film-air-bolshoj-pryzhok-2023/',
 'https://metarankings.ru/film-barbi-2023/']

In [117]:
# Фильмов 50
len(film_links)

50

- Готово, получили чуть менее 50 по числу фильмов на одной странице поиска.


- Остался последний момент. Когда мы скачаем все фильмы с текущей страницы, нам нужно будет каким-то образом забраться на соседнюю. На сайте это можно делать просто пролистывая страницу вниз, javascript-функции подтянут новые мемы на текущее окно, но сейчас трогать эти функции не хочется.

- Обычно, все параметры, которые мы устанавливаем на сайте для поиска. 
Если мы хотим получить первую порцию фильмов, мы должны будем обратиться к сайту по ссылке

                `https://metarankings.ru/best-movies-2023/page/1/`


- Если мы захотим получить вторую поцию с 50, нам придётся немного видоизменить ссылку, а именно заменить номер страницы на 2.


                `https://metarankings.ru/best-movies-2023/page/2/`

Таким незатейливым образом мы сможем пройтись по всем страницам.

Наконец, обернем в красивую функцию все-все манипуляции, проделанные выше:

In [118]:
def getPageLinks(page_number):
    """
        Возвращает список ссылок на фильмы, полученный с текущей страницы

        page_number: int/string
            номер страницы для парсинга
     Возвращает:
        list: список ссылок на мемы в формате 'https://metarankings.ru/...' или пустой список, если сервер отказал в доступе.
    """
    # составляем ссылку на страницу поиска
    page_link = 'https://metarankings.ru/best-movies-2023/page/{}'.format(page_number)

    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})

    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return []

    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # наконец, ищем ссылки на мемы и очищаем их от ненужных тэгов
    film_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['thumb'])
    film_links = ['https://metarankings.ru/' + link.attrs['href'] for link in film_links]

    return film_links

Протестируем функцию и убедимся, что всё хорошо

In [119]:
film_links = getPageLinks(1)
film_links[:2]

['https://metarankings.ru/https://metarankings.ru/film-solnce-moyo-2023/',
 'https://metarankings.ru/https://metarankings.ru/bednye-neschastnye-2023/']

In [120]:
meme_links = getPageLinks(2)
meme_links[:2]

['https://metarankings.ru/https://metarankings.ru/film-ostrov-illyuzij-2023/',
 'https://metarankings.ru/https://metarankings.ru/film-byuro-magicheskix-uslug-2023/']

-- Ой как хорошо, функция работает и теперь мы теоретически можем достать ссылки на все фильмы, для чего нам придется пройтись по 12+ страниц.
Прежде чем расстраивать сервер таким количеством запросов, посмотрим, как доставать всю необходимую информацию о конкретном фильме.

## 1.5 Финальная подготовка 

По аналогии со ссылками можно вытащить что угодно. Для этого надо сделать несколько шагов:

1. Открываем страничку с фильмом
2. Находим любым способом тег для нужной нам информации
3. Пихаем всё это в красивый суп
4. ......
5. Profit

Для закрепления информации в голове любознательного читателя, вытащим:
-текстовое описание фильма + средний рейтинг в качестве целевой переменной.
- тексты заметок + число просмотров.
- идеи, которые подходят под такой формат.
Сама страница, с которой мы будем доставать дорогую нашему исследовательскому сердцу информацию выглядит следуюшим образом:
https://metarankings.ru/m3gan/

Описание
Фильм «М3ГАН» — хоррор, который рассказывает о современном робототехнике в компании по производству игрушек по имени Джемми. В один прекрасный день она получает опеку над своей недавно осиротевшей племянницей и дарит девочке экспериментальную куклу-андроид «М3ГАН», в программу которой входит защита своей хозяйки, но вскоре она выходит из под контроля и начинает угрожать всем, кто подходит к девочке.

Как и прежде, для начала сохраним ссылку на страницу в переменную и вытащим по ней контент.

In [121]:
film_page = 'https://metarankings.ru/m3gan/'
film_page

'https://metarankings.ru/m3gan/'

In [122]:
film_page = 'https://metarankings.ru/m3gan/'

response = requests.get(film_page, headers={'User-Agent': UserAgent().chrome})

html = response.content
soup = BeautifulSoup(html,'html.parser')

Посмотрим, как можно вытащить всю вышеуказанную статистику:
-текстовое описание фильма + средний рейтинг в качестве целевой переменной.
- идеи, которые подходят под такой формат.

In [123]:
# <div class="rating"> <div class="score"> 7.1 </div> Оценки: <span>19</span> </div>

score = soup.find(attrs={'class':'score'})
score

<div class="score">
7.1
</div>

In [124]:
type(score)

bs4.element.Tag

Средняя оценка

In [125]:
score

<div class="score">
7.1
</div>

Описание

In [126]:
description = soup.find(attrs={'class':'description'})
description

<div class="description">
<h2>Описание</h2>
<div itemprop="description">
<p>Фильм «М3ГАН» — хоррор, который рассказывает о современном робототехнике в компании по производству игрушек по имени Джемми. В один прекрасный день она получает опеку над своей недавно осиротевшей племянницей и дарит девочке экспериментальную куклу-андроид «М3ГАН», в программу которой входит защита своей хозяйки, но вскоре она выходит из под контроля и начинает угрожать всем, кто подходит к девочке.</p>
</div>
<h2>Актеры</h2>
<p><a href="/aktery/эллисон_уильямс/" itemprop="actor">Эллисон Уильямс</a>, <a href="/aktery/вайолет_макгроу/" itemprop="actor">Вайолет Макгроу</a>, <a href="/aktery/эми_дональд/" itemprop="actor">Эми Дональд</a>, <a href="/aktery/дженна_дэвис/" itemprop="actor">Дженна Дэвис</a>, <a href="/aktery/джен_браун/" itemprop="actor">Джен Браун</a>, <a href="/aktery/брайан_джордан_альварез/" itemprop="actor">Брайан Джордан Альварез</a>, <a href="/aktery/джек_кэссиди/" itemprop="actor">Джек Кэссиди

In [127]:
type(description)

bs4.element.Tag

Отзывы

In [128]:
revs = soup.find(attrs={'class':'revs'})
revs

<span class="revs">Отзывы</span>

In [129]:
type(revs)

bs4.element.Tag

Рейтинг

In [130]:
rating_raph_mark_6 =  soup.find(attrs={'class':'rating-graph mark-6'})
rating_raph_mark_6

<div class="rating-graph mark-6" style="width:58%"><span class="score-graph">5.8</span></div>

In [131]:
type(rating_raph_mark_6)

bs4.element.Tag

Общий рейтинг

In [132]:
rating_raph_mark_7 =  soup.find(attrs={'class':'rating-graph mark-7'})
rating_raph_mark_7

<div class="rating-graph mark-7" style="width:71%"><span class="score-graph">7.1</span></div>

In [133]:
type(rating_raph_mark_7)

bs4.element.Tag

Ожидания

In [134]:
wait_stat =  soup.find(attrs={'class':'wait-stat'})
wait_stat

<div class="wait-stat">Ожидания: 56%</div>

In [135]:
type(wait_stat)

bs4.element.Tag

Преобразуем всё это в небольшую функцию.

In [136]:
def getStats(soup, stats):
 try:
    obj = soup.find('div', attrs={'class':stats})
    text = obj.text.strip()
    return text.replace("%", "") # Удаление символа '%'
 except:
    return None

description = getStats(soup, stats='description')
average_score = getStats(soup, stats='score')
critics_rating = getStats(soup, stats='rating-graph mark-6')

expectations = getStats(soup, stats='wait-stat') # добавить ожидания
span = soup.find('span', attrs={'class':'release-date'})
release_date = span.text # добавить дату выхода
#release_date = getStats(soup, stats='release-date') # добавить дату выхода
#release_date = release_date.text
print("Описание: {}\nСредняя оценка: {}\nРейтинг критиков: {}\nОжидания: {}\nДата выхода: {}".format(description, average_score, critics_rating, expectations, release_date))


Описание: Описание

Фильм «М3ГАН» — хоррор, который рассказывает о современном робототехнике в компании по производству игрушек по имени Джемми. В один прекрасный день она получает опеку над своей недавно осиротевшей племянницей и дарит девочке экспериментальную куклу-андроид «М3ГАН», в программу которой входит защита своей хозяйки, но вскоре она выходит из под контроля и начинает угрожать всем, кто подходит к девочке.

Актеры
Эллисон Уильямс, Вайолет Макгроу, Эми Дональд, Дженна Дэвис, Джен Браун, Брайан Джордан Альварез, Джек Кэссиди, Ронни Чиэн, Лори Дунги, Стефан Гарно, ...
Средняя оценка: 7.1
Рейтинг критиков: 5.8
Ожидания: Ожидания: 56
Дата выхода: 26 января 2023


In [137]:
#span = soup.find('span', attrs={'class':'release-date'})
#release_date = span.text
#print(date) # Outputs: 26 января 2023

Всё готово!

Наконец, создадим функцию, возвращающую всю информацию по фильму

In [138]:
def getMemeData(film_page):
    """
        Запрашивает данные по странице, возвращает обработанный словарь с данными

        meme_page: string
            ссылка на страницу с мемом

    """

    # запрашиваем данные по ссылке
    response = requests.get(film_page, headers={'User-Agent': UserAgent().chrome})

    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки
        return response.status_code

    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    description = getStats(soup, stats='description')
    average_score = getStats(soup, stats='score')
    critics_rating = getStats(soup, stats='rating-graph mark-6')

    expectations = getStats(soup, stats='wait-stat') # добавить ожидания
    release_date = getStats(soup, stats='release-date') # добавить дату выхода
    #release_date = span.text
    

    # составляем словарь, в котором будут хранится все полученные и обработанные данные
    data_row = {"release-date":release_date, "description":description,
                "score":score, "rating-graph mark-6":critics_rating , "wait-stat":expectations }
    #data_row.update(text_fileds)

    return data_row

In [139]:
data_row = getMemeData('https://metarankings.ru/m3gan/')

In [140]:
data_row

{'release-date': None,
 'description': 'Описание\n\nФильм «М3ГАН» — хоррор, который рассказывает о современном\xa0робототехнике в\xa0компании по\xa0производству игрушек по имени Джемми. В один прекрасный день она получает опеку над\xa0своей недавно осиротевшей племянницей и дарит девочке экспериментальную куклу-андроид «М3ГАН», в программу которой входит защита своей хозяйки, но вскоре она выходит из под контроля и начинает угрожать всем, кто подходит к девочке.\n\nАктеры\nЭллисон Уильямс, Вайолет Макгроу, Эми Дональд, Дженна Дэвис, Джен Браун, Брайан Джордан Альварез, Джек Кэссиди, Ронни Чиэн, Лори Дунги, Стефан Гарно, ...',
 'score': <div class="score">
 7.1
 </div>,
 'rating-graph mark-6': '5.8',
 'wait-stat': 'Ожидания: 56'}

А теперь подготовим табличку, чтобы в неё записывать всё ~~награбленные~~ честно полученные данные, добавим в неё первую полученную строку и полюбуемся на результат

In [141]:
#pip install --upgrade pandas


In [142]:
import pandas as pd
print(pd.__version__)



2.1.4


In [143]:
print("Описание: {}\nСредняя оценка: {}\nРейтинг критиков: {}\nОжидания: {}\nДата выхода: {}".format(description, average_score, critics_rating, expectations, release_date))


Описание: Описание

Фильм «М3ГАН» — хоррор, который рассказывает о современном робототехнике в компании по производству игрушек по имени Джемми. В один прекрасный день она получает опеку над своей недавно осиротевшей племянницей и дарит девочке экспериментальную куклу-андроид «М3ГАН», в программу которой входит защита своей хозяйки, но вскоре она выходит из под контроля и начинает угрожать всем, кто подходит к девочке.

Актеры
Эллисон Уильямс, Вайолет Макгроу, Эми Дональд, Дженна Дэвис, Джен Браун, Брайан Джордан Альварез, Джек Кэссиди, Ронни Чиэн, Лори Дунги, Стефан Гарно, ...
Средняя оценка: 7.1
Рейтинг критиков: 5.8
Ожидания: Ожидания: 56
Дата выхода: 26 января 2023


In [144]:
final_df = pd.DataFrame(columns=['description',  'dverage_score', 'critics_rating', 'expectations',
                                 'release_date'])

In [145]:
final_df = pd.concat([final_df, pd.DataFrame([data_row])], ignore_index=True)


In [146]:
final_df.head(5)

,description,dverage_score,critics_rating,expectations,release_date,release-date,score,rating-graph mark-6,wait-stat
0,"Описание\n\nФильм «М3ГАН» — хоррор, который ра...",NaN,NaN,NaN,NaN,None,[\r\n7.1\r\n],5.8,Ожидания: 56


 Еще раз убедимся что всё работает — пройдемся по списку из ссылок, полученных ранее в перменной `_links`.

In [147]:
#pip install tqdm

In [148]:
# Импортировать модуль tqdm.notebook для создания полос прогресса в Jupyter Notebook
from tqdm.notebook import tqdm


In [149]:
film_links

['https://metarankings.ru/https://metarankings.ru/film-solnce-moyo-2023/',
 'https://metarankings.ru/https://metarankings.ru/bednye-neschastnye-2023/',
 'https://metarankings.ru/https://metarankings.ru/dzhon-uik-4/',
 'https://metarankings.ru/https://metarankings.ru/film-strazhi-galaktiki-chast-3-2023/',
 'https://metarankings.ru/https://metarankings.ru/vonka-2023/',
 'https://metarankings.ru/https://metarankings.ru/film-posle-yanga-2023/',
 'https://metarankings.ru/https://metarankings.ru/ostavlennye-2023/',
 'https://metarankings.ru/https://metarankings.ru/film-po-pravilam-i-bez-2023/',
 'https://metarankings.ru/https://metarankings.ru/film-air-bolshoj-pryzhok-2023/',
 'https://metarankings.ru/https://metarankings.ru/film-barbi-2023/',
 'https://metarankings.ru/https://metarankings.ru/lunatiki-2023/',
 'https://metarankings.ru/https://metarankings.ru/podzemelya-i-drakony-chest-sredi-vorov/',
 'https://metarankings.ru/https://metarankings.ru/godzilla-minus-odin-2023/',
 'https://metar

In [150]:
for film_links in tqdm(film_links):
    try:
        data_row = getMemeData(film_links)
        final_df = final_df.append(data_row, ignore_index=True)
        time.sleep(0.3)
    except:
        continue

  0%|          | 0/50 [00:00<?, ?it/s]

In [151]:
final_df = final_df.drop_duplicates().dropna(axis = 1)

In [152]:
final_df.shape

(1, 4)

In [153]:
final_df.head()

,description,score,rating-graph mark-6,wait-stat
0,"Описание\n\nФильм «М3ГАН» — хоррор, который ра...",[\r\n7.1\r\n],5.8,Ожидания: 56


In [154]:
# Сохранить датафрейм в файл film_data.csv
final_df.to_csv('film_data.csv', index=False)

Отлично! Всё работает, мемы качаются, данные наполняются и всё было бы хорошо, если бы не одно но — количество запросов, которое нам придётся сделать, чтобы всё получить.

# 2.  Момент истины

Вот он! Тот самый момент абсолютного триумфа, когда код дописан и всё, что нам, мирным собирателям, остаётся — запустить наш код на одну ночку. Кажется, что через страсть мы преобрели силу. Запускаем наш код по всем  страницам с мемами. На всякий случай обернём наш цикл в `try-except`. Мало ли что там с этими мемами бывает.

In [155]:
import os
print(os.getcwd())


C:\Users\kolin\Downloads\Documents\Парсер данных


In [162]:
# Импортировать модули pandas и tqdm
import pandas as pd
from tqdm.notebook import tqdm

# Создать пустой датафрейм с нужными столбцами
final_df = pd.DataFrame(columns=['description', 'dverage_score', 'critics_rating', 'expectations',
                             'release_date'])

# Для каждого номера страницы от 1 до 12
for page_number in tqdm(range(1, 2), desc='Pages'):
  # Собрать ссылки на фильмы с текущей страницы
  film_links = getPageLinks(page_number)
  # Для каждой ссылки на фильм в списке film_links
  for film_link in tqdm(film_links, desc='Films', leave=False):
      # Иногда с первого раза страница не парсится
      for i in range(3):
          try:
              # Попытаться собрать данные о фильме
              data_row = getMemeData(film_link)  # оставил название переменной не изменял при желании можно изменить
              # И добавить их в датафрейм
              final_df = final_df.append(data_row, ignore_index=True)
              # Если все получилось - выйти из внутреннего цикла
              break
          except:
              # Иначе, пробовать еще несколько раз, пока не закончатся попытки
              print('AHTUNG! parsing once again:', film_link)
              continue
          # Добавить задержку между запросами
          time.sleep(0.3)


Pages:   0%|          | 0/1 [00:00<?, ?it/s]

Films:   0%|          | 0/50 [00:00<?, ?it/s]

AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/film-solnce-moyo-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/film-solnce-moyo-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/film-solnce-moyo-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/bednye-neschastnye-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/bednye-neschastnye-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/bednye-neschastnye-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/dzhon-uik-4/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/dzhon-uik-4/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/dzhon-uik-4/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/film-strazhi-galaktiki-chast-3-2023/
AHTUNG! parsing once

AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/film-ubijcy-cvetochnoj-luny-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/film-vse-straxi-bo-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/film-vse-straxi-bo-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/film-vse-straxi-bo-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/film-kokainovyj-medved-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/film-kokainovyj-medved-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/film-kokainovyj-medved-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/renfild-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/renfild-2023/
AHTUNG! parsing once again: https://metarankings.ru/https://metarankings.ru/renfild-2023/
AH

##### Сохраняем датафрейм в файл

In [159]:
# Сохранить датафрейм в файл film_data.csv
final_df.to_csv('film_data.csv', index=False)


https://metarankings.ru/robots.txt

In [168]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

In [169]:
checkIP()

Current IP Address: 88.150.230.100


Спасибо за внимание!